In [1]:
from typing import Generic, Optional, Tuple, List

import numpy as np
from torchvision.models import resnet50
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import utils
from utils import debugt, debugs, debug

torch.hub.set_dir('torch_cache')
import fishdetr_batchboy as detr
import contextlib
from generators import TorchStereoDataset
import re

In [2]:
utils.seed_everything(42069)

In [3]:
DATA_DIR = '/mnt/blendervol/leftright_left_data'
TABLE = 'bboxes_std'

In [10]:
debugt = utils.reloader(debugt)

traingen = TorchStereoDataset(DATA_DIR, TABLE, shuffle=False)
trainloader = torch.utils.data.DataLoader(traingen, batch_size=4, collate_fn=detr.collate)
X, y = next(iter(trainloader))

In [25]:
utils.reloader(detr)
model = detr.FishDETR()
model = model.eval()

lefts = X[0]
with torch.no_grad():
    output = model(X)
    debugs(output['pred_boxes'])

Encoder successfully loaded with pretrained weights
Encoder layers are frozen
(8, <module>) output['pred_boxes']: torch.Size([4, 100, 4])
